# O projeto
![Imagem do sistema](docs/system_proposal.png)

In [46]:
# feito por arthur lorencini
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [47]:
# Definindo os os caminhos
raw_dataset_path = Path("data/raw/")
wind_dataset_path = raw_dataset_path / 'wind/VentoCurvaPotencia.txt'
consuption_dataset_path = raw_dataset_path / 'consumption' / 'Brava_consumo.xlsx'
cases_dataset_path = raw_dataset_path / 'cases' / 'cases.csv'

In [48]:
# lendo o arquivo de texto que contém a quantidade de vento
df_wind = pd.read_csv(wind_dataset_path,sep='\t',header=None)
df_wind.columns = ['velocity','2','3','4']
# O que os números significam:
# Os números são para indicar 
# 2:ntk150, 3:ntk300, 4:acsa225
print(df_wind.head())
print("O dataset deve conter 4 colunas e 8760 linhas")
print(df_wind.shape)

   velocity     2     3     4
0       6.5  39.5  44.4  41.8
1       5.6  24.4   0.0  26.5
2       5.3  18.5   0.0  21.2
3       2.4   0.0   0.0   0.0
4       3.1   0.0   0.0   0.0
O dataset deve conter 4 colunas e 8760 linhas
(8760, 4)


In [49]:
# pegando os dados de consumo/demanda da ilha
# O dataset possui umas linhas que não vão agregar em nada
# Vamos apenas pegar as linhas
df_cons = pd.read_excel(consuption_dataset_path,sheet_name='Carga2010-2037',skiprows=1,nrows=8760)
if df_cons.shape[0] != 8760:
    raise ValueError("O dataset de consumo está com o tamanho errado")
df_cons.head()


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,...,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,252.0,259.56,267.3468,275.367204,283.628220,292.137067,277.959276,286.298054,294.886996,303.733606,...,396.303464,408.192568,420.438345,433.051495,446.043040,459.424331,473.207061,487.403273,502.025371,517.086132
1,238.0,245.14,252.4942,260.069026,267.871097,275.907230,262.517094,270.392607,278.504385,286.859517,...,374.286605,385.515203,397.080659,408.993079,421.262871,433.900757,446.917780,460.325313,474.135073,488.359125
2,228.0,234.84,241.8852,249.141756,256.616009,264.314489,251.486964,259.031573,266.802520,274.806596,...,358.560277,369.317085,380.396598,391.808496,403.562750,415.669633,428.139722,440.983914,454.213431,467.839834
3,225.0,231.75,238.7025,245.863575,253.239482,260.836667,248.177925,255.623263,263.291961,271.190719,...,353.842378,364.457650,375.391379,386.653121,398.252714,410.200296,422.506305,435.181494,448.236939,461.684047
4,220.0,226.60,233.3980,240.399940,247.611938,255.040296,242.662860,249.942746,257.441028,265.164259,...,345.979214,356.358591,367.049349,378.060829,389.402654,401.084734,413.117276,425.510794,438.276118,451.424401


Cada ano mostra o consumo da ilha de Brava, se eu não me engano. Cada ano possui 8760 horas

In [50]:
# Definindo constantes
# Eficiência média da bomba
avg_pump_eff = 0.86 # nb
# Eficiência média da turbina hidráulica (eu acho)
avg_hydraulic_turbine_efficiency = 0.86 # nt
# Eficiência média da conversão da potência eólica
avg_conversion_eff = 0.97 # nw

# O mínimo de potência de entrega?
minimum_power_output_from_diesel_generator = 256*0.5 # PDmin


In [51]:
# Definindo a matrix de resultados
df_result = pd.DataFrame({})


In [52]:
# Definindo os casos (cenários simulados)
# Esse csv eu criei para usar como banco de dados
df_cases = pd.read_csv(cases_dataset_path)
df_cases.head()

,aero_power_kW,number_of_aero,aero_id,reservatory_volume_m3
0,150,2,2,1500
1,300,1,3,1500
2,150,3,2,2250
3,225,2,4,2250
4,225,3,4,3400


aero_id se refere o id lá do dataset do vento.

![Calculo reservatório](docs/reservoir.png)

In [53]:
def resevartory_volume_to_energy(volume_m3:float):
    """recebe em m3 e devolve em kWh
     o quanto de energia um reservatório pode armazenar
     """
    return volume_m3/3600*9.8*110

In [54]:
# Aqui vamos calcular o quanto que cada combinação pode nos dar de energia inicial
df_cases['reservatory_stored_energy_kWh'] = resevartory_volume_to_energy(df_cases.reservatory_volume_m3)
df_cases.head()

,aero_power_kW,number_of_aero,aero_id,reservatory_volume_m3,reservatory_stored_energy_kWh
0,150,2,2,1500,449.166667
1,300,1,3,1500,449.166667
2,150,3,2,2250,673.750000
3,225,2,4,2250,673.750000
4,225,3,4,3400,1018.111111


In [55]:
df_cases['max_power'] = df_cases.aero_power_kW*df_cases.number_of_aero
df_cases.head()

,aero_power_kW,number_of_aero,aero_id,reservatory_volume_m3,reservatory_stored_energy_kWh,max_power
0,150,2,2,1500,449.166667,300
1,300,1,3,1500,449.166667,300
2,150,3,2,2250,673.750000,450
3,225,2,4,2250,673.750000,450
4,225,3,4,3400,1018.111111,675


![Algo](docs/algo.png)
![energy](docs/energy_wind.png)
![also energy](docs/energy_1.png)

In [56]:
"""
Legenda
Ewf is the total hourly produced energy of the wind farm;
P_w is the hourly power of a single wind turbine; 
n_w is the wind farm overall efficiency (97%); 
and N wind turbine units;

Ewf = n_w * N * P_w * 1 hora (kkkkk)
Ewf = Energia injetada na rede + energia entregue nas bombas, energia desperdiçada
Ewig = energia do vento na rede (Energy WInd Generated)
Ep = Energy Pump
E dis = Energy Dissipated

Essa energia desperdiçada acontece caso o reservatório esteja cheio.

Ed = Energia média consumida em uma hora (energia Demandada)
Ed = Energia do vento + energia do diesel + energia das turbinas hidráulicas
Ed = Ewig + Edg + Eh

Erj = Energia Inicial no Reservatório = current_stored_energy
Pwt = Power Wind Turbine - Potência do parque eólico
Pd = Power Demand (da rede da ilha)
Pt = 
Pb = 
Pge = 
Pwr = energia que o parque eólico injetou na rede (Ewig) 
Ptmax = potêcnia máxima da turbina hidráulica
PDmin = O grupo eletrogéneo não pode funcionar abaixo de 50% da carga
O menor grupo tem a potência de 250kW
PDmin = 256*0.5


O algoritmo tem como entrada os seguintes valores:
A demanda de energia - Ed
Velocidade do vento - Ws (?)
A curva de potência do gerador eólico
Capacidade máxima de armazenamento do reservatório E_r_max (energy reservoir max)

Temos que analisar até 5 condicionais
1º → A Energia do parque éolico é menor que 35% da demanda?
Se sim, A energia gerada do vento vai ser a mesma da demanda Ewf = Ewig (demanda tem que atender oferta)
2º Se a energia do vento que sobrou depois de injetar na rede for 
menor quanto o espaço que falta de preencher do reservatório
(Ex: Temos 10kWh do vento e ainda podemos colocar mais 20kWh no reservatório (que corresponde a um volume x))
Então, vc manda esse restante de energia pras bombas (Ep - Energy Pump)
Aí a energia do reservatório vai aumentar com o tempo
Se for maior (temos sobrando 30kWh do vento e falta 20kWh do reservatório),
você enche o reservatório até o talo fi kkkkkkkkkkk
O resto você dissipa nas resistências do universo

3º Se a diferença entre a demanda e o quanto injetou de vento for 
menor que que a energia do reservatório
Então vc roda o 4º
Se for maior




Note: Potência e Energia assumem o mesmo valor numérico
pois estamos tratando no período médio de uma hora

E = P * h, se Geramos 13kW (médios) em 1 h, então geramos 13kWh.

"""
injected_power_values = []
for idx, case_row in df_cases.iterrows():
    aero_power = case_row['aero_power_kW']
    number_of_aero = case_row['number_of_aero']
    aero_id = str(int(case_row['aero_id']))
    maximum_reservoir_energy = case_row['reservatory_stored_energy_kWh']
    current_reservoir_energy = maximum_reservoir_energy/2

    # eff = out/in → eff = elect/wind → wind = elect*eff
    # Criando a coluna de potência total do parque eólico
    combined_wind_turbine_power = number_of_aero * avg_conversion_eff * df_wind[aero_id] 
    # NOTE
    # Por algum motivo, esse é o valor da potência da turbina hidraulica.
    # Não entendi ainda.
    max_hydraulic_turbine_power = combined_wind_turbine_power.mean()

    for year in df_cons.columns:
        # Vamos checar para cada hora de cada ano
        for hour_idx, consumption in enumerate(df_cons[year]):
            # consumption é o consumo médio kWH em uma hora do ano.
            # Se a energia gerada pelo parque eólico for maior que a demanda
            # Vamos injetar no máximo 35% da demanda na rede sempre.
            # E injetar no mínimo o máximo que o parque eólico produzir
            if combined_wind_turbine_power[hour_idx] > consumption*0.35:
                wind_injected_energy = consumption*0.35 # TODO: Armazenar em uma matriz
                injected_power_values.append(wind_injected_energy)
                available_wind_energy_to_pump = (combined_wind_turbine_power[hour_idx] - wind_injected_energy)*avg_pump_eff
                available_reservoir_energy_capacity = maximum_reservoir_energy - current_reservoir_energy
                # Verifica se o reservatório tem capacidade para 
                if available_wind_energy_to_pump < available_reservoir_energy_capacity:
                    # Se sim, ele vai injetar o que sobrou nas turbinas
                    # E o reservatório vai encher um pouco, não vai ser até o talo.    
                    # NOTE: Talvez esteja faltando a eficiência aqui, não sei
                    pump_power = combined_wind_turbine_power[hour_idx] - wind_injected_energy
                    current_reservoir_energy += available_wind_energy_to_pump
                else:
                    # O reservatório vai encher até o talo e dissipar o resto
                    # cuidado com as unidades
                    pump_power = available_reservoir_energy_capacity/avg_pump_eff
                    current_reservoir_energy = maximum_reservoir_energy
                    rejected_energy = combined_wind_turbine_power[hour_idx] - wind_injected_energy
                    rejected_energy -= (maximum_reservoir_energy - current_reservoir_energy)/avg_pump_eff
            else:
                # Se a energia do parque eólico for menor que 35% da demanda
                # ele vai injetar o que tem mesmo e abraços.
                wind_injected_energy = combined_wind_turbine_power[hour_idx]
                reservoir_energy = current_reservoir_energy # eu acho que ele salva o valor final

            # %--------------- energia de compensacao: hidraulica ou diesel --------
            #
            #
            # Se a energia do reservatório for maior que a demanda abatida (demanda - vento na rede)
            # Terceiro condicional - averiguando energia
            # if (Pd(j,i)-Pwr(j,i)) <= Erj*nt %%%%verifica se reservatorio tem energia hidrica suficiente para complementar a energia eolica
            remaining_energy_needs = consumption - wind_injected_energy
            if remaining_energy_needs <= current_reservoir_energy*avg_hydraulic_turbine_efficiency:
                # Vamos averiguar as potências médias
                # if (Pd(j,i)-Pwr(j,i)) <= Ptmax  % e se a hidroturbina  tem potencia suficiente.
                can_hydraulic_turbine_meet_demand = remaining_energy_needs <= max_hydraulic_turbine_power

                # 256 é 128 x 2
                # ou seja, 128kW é 50% da potência nominal do gerador a diesel
                # 128kW é o número do fluxograma
                is_remaining_demand_within_hydraulic_and_diesel_capacity = remaining_energy_needs <= max_hydraulic_turbine_power + minimum_power_output_from_diesel_generator
                # Ou seja, se a potência
                if can_hydraulic_turbine_meet_demand:
                    # Se atende, a potência da turbina hidráulica é a mesma da demanda restante
                    # O gerador a diesel fica desligado
                    # A energia armazenada no reservatório cai
                    hidraulic_turbine_power = remaining_energy_needs
                    diesel_generation_power = 0
                elif is_remaining_demand_within_hydraulic_and_diesel_capacity:
                    # Ou seja, aqui já a turbina hidráulica não segura a buxa sozinha.
                    # Se a demanda restante for menor que a turbina hidraulica + o valor mínimo
                    # de entrega do gerador
                    # Então a gente liga o gerador a diesel no mínimo (128 kW)
                    # A potencia da turbina vai ficar pra abastecer o que restou
                    # E a energia do reservatório cai
                    diesel_generation_power = minimum_power_output_from_diesel_generator
                    hidraulic_turbine_power = remaining_energy_needs - diesel_generation_power
                else:
                    # O elif do .m virou else aqui
                    # Se a demanda restante for maior que a potencia minima + potência máxima da turbina hidraulica
                    # Liga a turbina no talo
                    # Liga o gerador pra abastecer o restante
                    # e a energia no reservatório vai caindo
                    hidraulic_turbine_power = max_hydraulic_turbine_power
                    diesel_generation_power = remaining_energy_needs - hidraulic_turbine_power
                # Essa condição é comum para os tres casos do .m
                current_reservoir_energy -= hidraulic_turbine_power/avg_hydraulic_turbine_efficiency

            # Se vai faltar água no reservatório pra antender a demanda faltante
            # elseif (Pd(j,i)-Pwr(j,i)) > Erj*nt %%%%%verifica se reservatorio tem energia hidrica suficiente para complementar a energia eolica. A turbina tb nao pode atender tudo
            else:
                # o elseif do verifica lá virou else aqui por razões lógicas.
                # Se a energia armazenada está entre a potência máxima da turbina + entrega minima e 
                # a potência máxima da turbin
                # if Erj*nt > Ptmax  && Erj*nt < ( Ptmax + PDmin) 
                if (max_hydraulic_turbine_power + minimum_power_output_from_diesel_generator > current_reservoir_energy*avg_hydraulic_turbine_efficiency and
                    current_reservoir_energy*avg_hydraulic_turbine_efficiency > max_hydraulic_turbine_power):
                    is_hydraulic_power_sufficient = max_hydraulic_turbine_power >= minimum_power_output_from_diesel_generator and max_hydraulic_turbine_power >= remaining_energy_needs
                    # if Ptmax >= PDmin && (Pd(j,i)-Pwr(j,i))<= Ptmax % garante que Turbina fica abaixo da sua potencia
                    if is_hydraulic_power_sufficient:
                        diesel_generation_power = consumption
                        hidraulic_turbine_power = remaining_energy_needs - minimum_power_output_from_diesel_generator
                        current_reservoir_energy -= hidraulic_turbine_power/avg_hydraulic_turbine_efficiency
                    else:
                        diesel_generation_power = remaining_energy_needs
                elif (current_reservoir_energy*avg_hydraulic_turbine_efficiency > max_hydraulic_turbine_power + minimum_power_output_from_diesel_generator):
                    hidraulic_turbine_power = max_hydraulic_turbine_power
                    diesel_generation_power = remaining_energy_needs - hidraulic_turbine_power
                    current_reservoir_energy -= hidraulic_turbine_power/avg_hydraulic_turbine_efficiency
                else:
                    diesel_generation_power = remaining_energy_needs